In [67]:
import cv2
import os
import time

image_dir = r'C:\Users\front\Downloads\archive\Original Images\Original Images' # ローカルの画像データファイルパス


In [68]:
processed_images = []
labels = []
label_name = []
haarcascade_path = r'data\haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(haarcascade_path)
print("Using Haar Cascade path:", haarcascade_path)
if not os.path.exists(haarcascade_path):
    raise FileNotFoundError(f'Haar Cascade XML file not found at {haarcascade_path}')
face_cascade = cv2.CascadeClassifier(haarcascade_path)

Using Haar Cascade path: data\haarcascade_frontalface_default.xml


In [69]:
for label, person_name in enumerate(os.listdir(image_dir)):
    person_dir = os.path.join(image_dir, person_name)
    if not os.path.isdir(person_dir):
        continue

    for image_name in os.listdir(person_dir):
        image_path = os.path.join(person_dir, image_name)
        image = cv2.imread(image_path)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5)

        for (x, y, w, h) in faces:
            face = gray_image[y:y+h, x:x+w]
            resized_face = cv2.resize(face, (128, 128))
            processed_images.append(resized_face)
            labels.append(label)
            label_name.append(person_name)

In [70]:
import tensorflow as tf
from keras import layers
from keras import models
import numpy as np

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(set(labels)), activation='softmax')  # Number of unique labels
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

processed_images = np.array(processed_images).reshape(-1, 128, 128, 1)
labels = np.array(labels)

model.fit(processed_images, labels, epochs=10, validation_split=0.2)

model.save(r'C:\Users\front\OneDrive\ドキュメント\imageAI.h5')

Epoch 1/10


c:\Users\front\OneDrive\ドキュメント\imageAI\jupyter\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


105/105 ━━━━━━━━━━━━━━━━━━━━ 16s 147ms/step - accuracy: 0.0435 - loss: 28.9248 - val_accuracy: 0.0000e+00 - val_loss: 3.7371
Epoch 2/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 15s 146ms/step - accuracy: 0.1216 - loss: 3.2378 - val_accuracy: 0.0012 - val_loss: 5.4759
Epoch 3/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 15s 144ms/step - accuracy: 0.2952 - loss: 2.5282 - val_accuracy: 0.0048 - val_loss: 8.3200
Epoch 4/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 15s 144ms/step - accuracy: 0.4723 - loss: 1.8852 - val_accuracy: 0.0095 - val_loss: 13.8149
Epoch 5/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 15s 146ms/step - accuracy: 0.6022 - loss: 1.4318 - val_accuracy: 0.0155 - val_loss: 20.5649
Epoch 6/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 15s 147ms/step - accuracy: 0.7241 - loss: 0.9865 - val_accuracy: 0.0084 - val_loss: 25.5908
Epoch 7/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 16s 149ms/step - accuracy: 0.7657 - loss: 0.8680 - val_accuracy: 0.0084 - val_loss: 35.4864
Epoch 8/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 15s 147ms/step - accuracy: 0.7972 - loss: 0.7

In [71]:
def recognize_face(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face = gray_image[y:y+h, x:x+w]
        resized_face = cv2.resize(face, (128, 128)).reshape(1, 128, 128, 1)
        predictions = model.predict(resized_face)
        label = np.argmax(predictions)
        confidence = np.max(predictions)

        return label, confidence

    return None, None

In [73]:
video_capture = cv2.VideoCapture(0)
Absence = None
name = None
while True:
    ret, frame = video_capture.read()
    label, confidence = recognize_face(frame)

    # if label == 0:
    #     name = "Yudai"
    if confidence == None:
        if Absence == None:
            Absence = time.time()
        elif time.time() - Absence >= 5:
            break
    elif confidence > 0.7:  # しきい値
        Absence = None
        print(f"検出した人物名: {label_name[label]}")
        print(f"一致率: {confidence}")

video_capture.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
検出した人物名: Akshay Kumar
一致率: 0.9262316823005676
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
検出した人物名: Akshay Kumar
一致率: 0.9348993897438049
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
検出した人物名: Akshay Kumar
一致率: 0.7375233173370361
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
検出した人物名: Akshay Kumar
一致率: 0.9749271869659424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━